In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install datasets
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate
from transformers import AutoTokenizer , AutoModelForSeq2SeqLM
md="czearing/article-title-generator"
tokenizer=AutoTokenizer.from_pretrained(md)
model=AutoModelForSeq2SeqLM.from_pretrained(md)
import tensorflow as tf

Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
Found existing installation: accelerate 0.24.1
Uninstalling accelerate-0.24.1:
  Successfully uninstalled accelerate-0.24.1
  Using cached transformers-4.35.2-py3-none-any.whl (7.9 MB)
  Using cached accelerate-0.24.1-py3-none-any.whl (261 kB)


In [ ]:
from datasets import load_dataset
#dt=pd.read_csv('arxiv_data.csv')
dt=load_dataset('/content',data_files='arxiv_data.csv')
dt
dataset = dt['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dt['train']
k=dataset.train_test_split(test_size=0.2)
del k['test']
fresh=k['train'].train_test_split(test_size=0.2)
fresh

DatasetDict({
    train: Dataset({
        features: ['titles', 'summaries', 'terms'],
        num_rows: 33135
    })
    test: Dataset({
        features: ['titles', 'summaries', 'terms'],
        num_rows: 8284
    })
})

In [ ]:
def convert_examples_to_features(example_batch):
    input_encoding = tokenizer(example_batch['summaries'] ,padding="max_length", truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encoding = tokenizer(example_batch['titles'],padding="max_length", truncation = True )


    return{
        'input_ids' : input_encoding['input_ids'],
        'attention_mask': input_encoding['attention_mask'],
        'labels': target_encoding['input_ids']
    }

In [ ]:
dt_n = fresh.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/33135 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/8284 [00:00<?, ? examples/s]

In [ ]:
dt_n['train']

Dataset({
    features: ['titles', 'summaries', 'terms', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 33135
})

In [ ]:
#x_train,x_test,y_train,y_test=train_test_split(input_encodings,target_encodings,test_size=0.2,random_state=0)
#train_dt=tf.data.Dataset.from_tensor_slices((dict(x_train),y_train))
sample_text=dt_n['test'][0]['summaries']
dt_n['test'][0]['titles']

'Spatio-Temporal Convolutional LSTMs for Tumor Growth Prediction by Learning 4D Longitudinal Patient Data'

In [ ]:
#!pip install transformers['torch']
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='result', num_train_epochs=2, warmup_steps=500,
    per_device_train_batch_size=4, per_device_eval_batch_size=8,
    weight_decay=0.01, logging_steps=10,
    logging_dir='log',gradient_accumulation_steps=16,save_steps=1e6
)

In [ ]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
trainer = Trainer(model=model, args=training_args,
                  tokenizer=tokenizer,data_collator=seq2seq_data_collator,
                  train_dataset=dt_n['train'],
                  eval_dataset=dt_n['test']
                   )

In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
model.save_pretrained("Title_Generator")

In [ ]:
tokenizer.save_pretrained("Tokenizer")

('Tokenizer/tokenizer_config.json',
 'Tokenizer/special_tokens_map.json',
 'Tokenizer/tokenizer.json')

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('/content/Tokenizer')

In [ ]:
from transformers import pipeline
gen_kwargs={"length_penalty":0.8,"num_beams":8,"max_length":60,"min_length":10}
sample_text=dt_n['test'][5]['summaries']
title=dt_n['test'][5]['titles']
pipe=pipeline("summarization",model="Title_Generator",tokenizer=tokenizer)
print('Summary:')
print(sample_text)
print('Title:')
print(title)
print("Model Summary:")
print(pipe(sample_text,**gen_kwargs)[0]["summary_text"])

Summary:
This paper addresses the task of unsupervised learning of representations for
action recognition in videos. Previous works proposed to utilize future
prediction, or other domain-specific objectives to train a network, but
achieved only limited success. In contrast, in the relevant field of image
representation learning, simpler, discrimination-based methods have recently
bridged the gap to fully-supervised performance. We first propose to adapt two
top performing objectives in this class - instance recognition and local
aggregation, to the video domain. In particular, the latter approach iterates
between clustering the videos in the feature space of a network and updating it
to respect the cluster with a non-parametric classification loss. We observe
promising performance, but qualitative analysis shows that the learned
representations fail to capture motion patterns, grouping the videos based on
appearance. To mitigate this issue, we turn to the heuristic-based IDT
descriptor